In [3]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchResults

llm=ChatOpenAI(temperature=0.1)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)

class CompanyOverviewToolArgsSchema(BaseModel):
    symbol : str = Field(
        description="Stock symbol of the company. Example: AAPL, TSLA",
    )

class CompanyOverviewTool(BaseTool):
    name="CompanyOverview"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name="CompanyIncomeStatement"
    description="""
    Use this to get the income statement of company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools = [
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
    ],
)

# prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."
prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)




> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare'}`


[snippet: Real-time Cloudflare stock price (NYSE: NET) plus other valuable data points like day range, ... Market Cap Microsoft Corp. MSFT Exemplary: Wide: $3.2 Tril: Oracle Corp. ORCL Poor: Narrow:, title: Cloudflare Stock Price - NYSE: NET - Morningstar, link: https://www.morningstar.com/stocks/xnys/net/quote], [snippet: Cloud connectivity company Cloudflare Inc NET reported second-quarter revenue of $401.0 million, up 30% year-over-year. The revenue beat a Street consensus estimate of $394.69 million, according to da... 3 months ago - Benzinga., title: Cloudflare, Inc. (NET) Stock Price, Quote & News - Stock Analysis, link: https://stockanalysis.com/stocks/net/], [snippet: This is a decrease of -69% compared to the previous 30 days. In the past three months, Cloudflare insiders have sold more of their company's stock than they have bought. Specifi

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "### Cloudflare Inc (NET) Financial Information:\n\n- **Description:** CloudFlare, Inc. operates a cloud platform that offers a range of network services to companies around the world.\n- **Market Capitalization:** $29.92 billion\n- **Revenue (TTM):** $1.48 billion\n- **Gross Profit (TTM):** $742.63 million\n- **Net Income Margin (TTM):** -6.9%\n- **Operating Margin (TTM):** -8.65%\n- **Return on Assets (TTM):** -3.59%\n- **Return on Equity (TTM):** -13.4%\n- **EPS (TTM):** -$0.30\n- **Price to Sales Ratio (TTM):** 20.25\n- **Price to Book Ratio:** 33.94\n- **52-Week High:** $116\n- **52-Week Low:** $61.34\n- **Analyst Target Price:** $92.14\n- **Forward P/E Ratio:** 104.17\n\n### Income Statement (Annual):\n- **2023 Total Revenue:** $1.30 billion\n- **2023 Gross Profit:** $989.74 million\n-